In [1]:
import tensorflow as tf

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
        
def read_my_file_format(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    
    record_defaults = [[1] for i in range(541)]
#     print(record_defaults)

    cols = tf.decode_csv(value, record_defaults=record_defaults)
    labels = tf.one_hot(cols[1], 2)
    features = tf.stack(cols[2:])

    return features, labels

def input_pipeline(filenames, batch_size, num_epochs=None):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs, shuffle=True)
    example, label = read_my_file_format(filename_queue)
  # min_after_dequeue defines how big a buffer we will randomly sample
  #   from -- bigger means better shuffling but slower start up and more
  #   memory used.
  # capacity must be larger than min_after_dequeue and the amount larger
  #   determines the maximum we will prefetch.  Recommendation:
  #   min_after_dequeue + (num_threads + a small safety margin) * batch_size
    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batch = tf.train.shuffle_batch(
        [example, label], batch_size=batch_size, capacity=capacity,
        min_after_dequeue=min_after_dequeue)
    return example_batch, label_batch

with tf.name_scope('CSV_data'):
    with tf.name_scope('train_approved'):
        train_features_app, train_labels_app = input_pipeline(["data/train_approved.csv"], 200)
    with tf.name_scope('train_rejected'):
        train_features_rej, train_labels_rej = input_pipeline(["data/train_rejected.csv"], 100)
    with tf.name_scope('train_features'):
        train_features = tf.concat([train_features_app, train_features_rej], 0)
    with tf.name_scope('train_labels'):
        train_labels = tf.concat([train_labels_app, train_labels_rej], 0)

# train_features, train_labels = input_pipeline(["data/train_approved.csv", "data/train_rejected.csv"], 100)

    with tf.name_scope('test_data'):
        test_features, test_labels = input_pipeline(["data/test.csv"], 100)


init = tf.global_variables_initializer()

# with tf.Session() as sess:
#     sess.run(init)
    
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)

#     print( sess.run([train_features, train_labels]) )
    
#     coord.request_stop()
#     coord.join(threads)
    
# print("\nEnd.\n")


In [2]:

#define place holder
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 539])
    y = tf.placeholder(tf.float32, [None, 2])


#create a 3-level neural network
with tf.name_scope('Layers'):
    with tf.name_scope('Layers1'):
        with tf.name_scope('weights1'):
            W1 = tf.Variable(tf.truncated_normal([539, 300]))
            variable_summaries(W1)
        with tf.name_scope('bias1'):
            b1 = tf.Variable(tf.zeros([300])+0.1)
            variable_summaries(b1)
        with tf.name_scope('wx_b1'):
            wx_b1 = tf.matmul(x, W1)+b1
            variable_summaries(wx_b1)
        with tf.name_scope('activate_func1'):
            L1 = tf.nn.relu(wx_b1)
            variable_summaries(L1)

    with tf.name_scope('Layers2'):
        with tf.name_scope('weights2'):
            W2 = tf.Variable(tf.truncated_normal([300, 2]))
            variable_summaries(W2)
        with tf.name_scope('bias1'):
            b2 = tf.Variable(tf.zeros([2])+0.1)
            variable_summaries(b2)
        with tf.name_scope('wx_b2'):
            wx_b2 = tf.matmul(L1, W2)+b2
            variable_summaries(wx_b2)
        with tf.name_scope('activate_func2'):
            prediction = tf.nn.softmax(wx_b2)

with tf.name_scope('train'):
    with tf.name_scope('learning_rate'):
        #学习率使用指数衰减法
        global_step = tf.Variable(0)
        learn_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.95, staircase=True)
        tf.summary.scalar('learning_rate', learn_rate)
        
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
        # loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=prediction))
        tf.summary.scalar('loss', loss)
        
    with tf.name_scope('train'):
        #train
        train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    with tf.name_scope('Init'):
        init = tf.global_variables_initializer()

    with tf.name_scope('accuracy'):
        #比较预测值与实际值
        correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax 返回沿着某个维度最大值的位置

        #准确率
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

with tf.name_scope('Summary'):
    with tf.name_scope('train_sum'):
        y_sum = tf.reduce_sum(y, 0)
        variable_summaries(y_sum)

    with tf.name_scope('prediction_sum'):
        # pred_one_hot = tf.stack(tf.argmin(prediction, 1), tf.argmax(prediction, 1))
        # a = tf.argmin(prediction, 1)
        # b = tf.argmax(prediction, 1)
        # pred_one_hot = tf.transpose(tf.stack([a, b]))
        pred_one_hot = tf.one_hot(tf.argmax(prediction, 1), 2)
        # pred_one_hot = tf.one_hot(tf.argmax(prediction, 1), tf.shape(prediction)[0])
        print(pred_one_hot)

        pred_sum = tf.reduce_sum(pred_one_hot, 0)
    
    with tf.name_scope('intersection_sum'):
        pred_int_one_hot = tf.one_hot(tf.argmax(prediction, 1), 2, off_value=2.0)
        inter = tf.equal(y, pred_int_one_hot)
        print(inter)

        int_sum = tf.reduce_sum(tf.cast(inter, tf.int32), 0)

        # correct_rej = tf.equal(tf.argmax(y, 1), tf.argmax(pred_one_hot, 1))
        # int_rej_num = tf.reduce_sum(tf.cast(correct_rej, tf.int32))

        # correct_app = tf.equal(tf.argmin(y, 1), tf.argmin(pred_one_hot, 1))
        # int_app_num = tf.reduce_sum(tf.cast(correct_app, tf.int32))

merged = tf.summary.merge_all()


Tensor("Summary/prediction_sum/one_hot:0", shape=(?, 2), dtype=float32)
Tensor("Summary/intersection_sum/Equal:0", shape=(?, 2), dtype=bool)


In [3]:

with tf.Session() as sess:
    sess.run(init)
    
    writer = tf.summary.FileWriter('logs/', sess.graph)
    
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for step in range(121):
        # Retrieve a single instance:
        batch_xs, batch_ys = sess.run([train_features, train_labels])
#         print(batch_xs.shape, batch_ys.shape)
#         print(batch_xs)
    
        summary, _ = sess.run([merged, train], feed_dict={x: batch_xs, y: batch_ys})
#         sess.run(train, feed_dict={x: batch_xs, y: batch_ys})

        if step % 10 == 0:
            writer.add_summary(summary, step)
            
            acc_train = sess.run(accuracy, feed_dict={x:batch_xs, y:batch_ys})
            
            test_xs, test_ys = sess.run([test_features, test_labels])
            pred, acc_test = sess.run([prediction, accuracy], feed_dict={x:test_xs, y:test_ys})
            print("step:" + str(step) + " train accurancy:" + str(acc_train) + "  test accurancy:" + str(acc_test))
            
            y_s, p_s, i_s = sess.run([y_sum, pred_sum, int_sum], feed_dict={x:batch_xs, y:batch_ys} )
            print("Train - Label sum:" + str(y_s) + " Prediction sum" + str(p_s) + " Correct sum:" + str(i_s))
        
            y_s, p_s, i_s = sess.run([y_sum, pred_sum, int_sum], feed_dict={x:test_xs, y:test_ys} )
            print("Test - Label sum:" + str(y_s) + " Prediction sum" + str(p_s) + " Correct sum:" + str(i_s))
            print("")
                  
    coord.request_stop()
    coord.join(threads)

step:0 train accurancy:0.48666668  test accurancy:0.32
Train - Label sum:[200. 100.] Prediction sum[112. 188.] Correct sum:[79 67]
Test - Label sum:[93.  7.] Prediction sum[29. 71.] Correct sum:[27  5]

step:10 train accurancy:0.45  test accurancy:0.39
Train - Label sum:[200. 100.] Prediction sum[105. 195.] Correct sum:[70 65]
Test - Label sum:[88. 12.] Prediction sum[41. 59.] Correct sum:[34  5]

step:20 train accurancy:0.46  test accurancy:0.24
Train - Label sum:[200. 100.] Prediction sum[118. 182.] Correct sum:[78 60]
Test - Label sum:[95.  5.] Prediction sum[23. 77.] Correct sum:[21  3]

step:30 train accurancy:0.48333332  test accurancy:0.34
Train - Label sum:[200. 100.] Prediction sum[121. 179.] Correct sum:[83 62]
Test - Label sum:[90. 10.] Prediction sum[34. 66.] Correct sum:[29  5]

step:40 train accurancy:0.43333334  test accurancy:0.37
Train - Label sum:[200. 100.] Prediction sum[126. 174.] Correct sum:[78 52]
Test - Label sum:[93.  7.] Prediction sum[36. 64.] Correct sum:[3